# IMPORT

In [1]:
from __future__ import annotations
from bqskit import Circuit, compile
from bqskit.compiler.machine import MachineModel
from bqskit.ir.gates import RXXGate, RXGate, RZGate,RYGate,RZZGate,HGate,CNOTGate
from Custom_gate_decomposition.BQSkit_custom_gate_definitions import VirtualZGate, GPIGate, GPI2Gate,PartialMSGate,FullMSGate
from Custom_gate_decomposition.QASM2_reader import load_qasm
from bqskit.ext import bqskit_to_qiskit
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit import transpile
from qiskit.transpiler import Target, InstructionProperties
import pennylane as qml

     ┌────────────────────────┐┌─────────┐┌──────────────────────────────────┐»
q_0: ┤0                       ├┤ Gpi2(1) ├┤0                                 ├»
     │  Partialms(5,6,2.8047) │├─────────┤│  Partialms(3.8334,4.027,0.21048) │»
q_1: ┤1                       ├┤ Gpi2(5) ├┤1                                 ├»
     └────────────────────────┘└─────────┘└──────────────────────────────────┘»
«      ┌──────────────┐ ┌──────────────┐  ┌─────────┐  
«q_0: ─┤ Gpi2(3.3237) ├─┤ Gpi2(5.3237) ├──┤ Gpi2(4) ├──
«     ┌┴──────────────┤┌┴──────────────┤┌─┴─────────┴─┐
«q_1: ┤ Gpi2(0.34723) ├┤ Gpi2(0.15752) ├┤ Gpi2(3.435) ├
«     └───────────────┘└───────────────┘└─────────────┘


In [2]:
# Step 1: Define the MS-Gate Machine Model
IONQ_gate_set={PartialMSGate(),GPIGate(),GPI2Gate()}
def transpile_to_ms(circuit: Circuit) -> Circuit:
    """Transpile the given circuit to use only MS gates."""
    model = MachineModel(circuit.num_qudits, gate_set=IONQ_gate_set)
    compiled_circuit = compile(circuit, model, optimization_level=3)
    return compiled_circuit

# Step 2: Create a simple quantum circuit
qc = Circuit(2)
qc.append_gate(HGate(), [0])
qc.append_gate(RZZGate(), [0, 1],params=[1])
qc.append_gate(RYGate(), [1],params=[3])
qc.append_gate(RXXGate(), [0,1],params=[2.14584545])
qc.append_gate(RZZGate(), [1,0],params=[1.1561])
qc.append_gate(RXGate(), [1],params=[1.1561])
qc_qiskit = bqskit_to_qiskit(qc)

# Step 3: Transpile the circuit to MS gates
ms_circuit = transpile_to_ms(qc)

In [9]:
#SAVE QASM
ms_circuit.save("circuit.qasm")
print(qc_qiskit.draw(output='text'))

NameError: name 'ms_circuit' is not defined

In [2]:
# Step 4: Transform to qiskit and print
qiskit_circuit = load_qasm("circuit.qasm")
print(qiskit_circuit.draw(output='text'))


     ┌────────────────────────┐┌─────────┐┌──────────────────────────────────┐»
q_0: ┤0                       ├┤ Gpi2(1) ├┤0                                 ├»
     │  Partialms(5,6,2.8047) │├─────────┤│  Partialms(3.8334,4.027,0.21048) │»
q_1: ┤1                       ├┤ Gpi2(5) ├┤1                                 ├»
     └────────────────────────┘└─────────┘└──────────────────────────────────┘»
«      ┌──────────────┐ ┌──────────────┐  ┌─────────┐  
«q_0: ─┤ Gpi2(3.3237) ├─┤ Gpi2(5.3237) ├──┤ Gpi2(4) ├──
«     ┌┴──────────────┤┌┴──────────────┤┌─┴─────────┴─┐
«q_1: ┤ Gpi2(0.34723) ├┤ Gpi2(0.15752) ├┤ Gpi2(3.435) ├
«     └───────────────┘└───────────────┘└─────────────┘


# NOISE

In [4]:
# Convert Qiskit circuit to PennyLane
qml_circuit = qml.from_qiskit(qiskit_circuit)

In [13]:
qml.draw(qml_circuit)()

'0: ─╭U(M0)──U(M1)─╭U(M2)──U(M3)──U(M3)──U(M1)─┤  \n1: ─╰U(M0)──U(M1)─╰U(M2)──U(M4)──U(M5)──U(M6)─┤  \nM0 = \n[[-8.17616727e-01+0.j          0.00000000e+00+0.j\n   0.00000000e+00+0.j          8.86203872e-08-0.57576287j]\n [ 0.00000000e+00+0.j         -8.17616727e-01+0.j\n  -2.26748015e-08-0.57576287j  0.00000000e+00+0.j        ]\n [ 0.00000000e+00+0.j          2.26748015e-08-0.57576287j\n  -8.17616727e-01+0.j          0.00000000e+00+0.j        ]\n [-8.86203872e-08-0.57576287j  0.00000000e+00+0.j\n   0.00000000e+00+0.j         -8.17616727e-01+0.j        ]]\nM1 = \n[[ 7.07106781e-01+0.j          2.88388704e-07-0.70710678j]\n [-2.88388704e-07-0.70710678j  7.07106781e-01+0.j        ]]\nM2 = \n[[ 0.78923352+0.j          0.        +0.j          0.        +0.j\n   0.47214881-0.39266519j]\n [ 0.        +0.j          0.78923352+0.j          0.57587337-0.21326111j\n   0.        +0.j        ]\n [ 0.        +0.j         -0.57587337-0.21326111j  0.78923352+0.j\n   0.        +0.j        ]\n [-0.4721

In [37]:


# Создаем шумовую модель
noise_model = NoiseModel()

# 3% деполяризационная ошибка для PartialMS
partialMS_noise = depolarizing_error(1, 2)

# Добавляем базовые гейты (ВАЖНО: unitary fallback)
noise_model.add_basis_gates(["unitary"])
noise_model.add_all_qubit_quantum_error(partialMS_noise, ["partialMS"])

# Создаем Target с явным указанием кубитов
target = Target(num_qubits=2)

# Добавляем 1-кубитные гейты
target.add_instruction(GPIGate(0), {(0,): InstructionProperties(duration=1.0), (1,): InstructionProperties(duration=1.0)})
target.add_instruction(GPI2Gate(0), {(0,): InstructionProperties(duration=1.0), (1,): InstructionProperties(duration=1.0)})
target.add_instruction(VirtualZGate(0), {(0,): InstructionProperties(duration=0.5), (1,): InstructionProperties(duration=0.5)})

# Добавляем 2-кубитные гейты
target.add_instruction(PartialMSGate(0, 0, 0), {(0, 1): InstructionProperties(duration=2.0)})
target.add_instruction(FullMSGate(0, 0), {(0, 1): InstructionProperties(duration=2.0)})

# Создаем симулятор с кастомными гейтами и unitary fallback
simulator = AerSimulator(
    noise_model=noise_model,
    target=target,
    basis_gates=["unitary", "partialMS", "gpi", "gpi2"],  # Добавляем unitary
    custom_unitary={PartialMSGate()}  # Включаем поддержку произвольных унитарных операций
)

# Транспиляция схемы
tqc = transpile(qiskit_circuit, simulator, basis_gates=["unitary", "partialMS", "gpi", "gpi2"])

print(tqc.draw(output='text'))

# Запуск симуляции
result = simulator.run(tqc).result()

# Вывод результатов
print(result.get_counts())


AerError: 'Invalid option custom_unitary'

In [29]:




# Проверяем, какие гейты зарегистрированы
print("Registered instructions in target:", list(target.keys()))


Registered instructions in target: ['GPIGate', 'GPI2Gate', 'VirtualZGate', 'PartialMSGate', 'FullMSGate']
